In [1]:
import os
from dotenv import load_dotenv
import gradio as gr
import json
from openai import OpenAI

In [3]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
api_key = os.getenv('OR_API_KEY')
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [4]:
MODEL = 'openai/gpt-oss-20b:free'
openai = OpenAI(
    base_url='https://openrouter.ai/api/v1',
    api_key=api_key
)


In [5]:
system_message = "You are a helpful assistant for an Airline called FlightAI. Give Short responses and be accurate. And if you don't know the answer, say so. Do not make up answers."


In [6]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.


In [7]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [9]:
get_ticket_price("tokyo")

Tool get_ticket_price called for tokyo


'$1400'

In [11]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}
tools = [{"type": "function", "function": price_function}]

In [15]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response= handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [14]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    return {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
import base64
from io import BytesIO
from PIL import Image

In [ ]:
from gradio_client import Client

client = Client("ehristoforu/dalle-3-xl-lora-v2")

result = client.predict(
    "An image representing a vacation in Paris, vibrant pop-art style",
    "(deformed, distorted, disfigured:1.3), poorly drawn, bad anatomy...",
    True, 0, 1024, 1024, 6, True,
    fn_index=3
)

print(result)
# → [{'image': '/tmp/gradio/...png', 'caption': None}]


Loaded as API: https://ehristoforu-dalle-3-xl-lora-v2.hf.space ✔
([{'image': '/tmp/gradio/5601397f0343747aba66f40aba92441da865ffcfb3e9abf0e0b626fe9139815e/92f5468d-5101-4750-818a-bff975d090c9.png', 'caption': None}], 1593737565)


TypeError: list indices must be integers or slices, not str

In [20]:
image_url = "https://ehristoforu-dalle-3-xl-lora-v2.hf.space/file=" + result[0][0]['image']
print(image_url)


https://ehristoforu-dalle-3-xl-lora-v2.hf.space/file=/tmp/gradio/5601397f0343747aba66f40aba92441da865ffcfb3e9abf0e0b626fe9139815e/92f5468d-5101-4750-818a-bff975d090c9.png
